# Def test

In [ ]:
#Load dataset
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

In [ ]:
import scipy.stats

def ftest_5x2cv(df1, df2):
  #df1 df2 results table for each algorithm

    r2_variances = []
    r2_differences = []
    rmse_variances = []
    rmse_differences = []

    ####TEST RMSE

    for i in range(5):
        #first column==0 is R2
        #second column==1 is RMSE
        score_diff_1 = df1.iloc[i*2, 1]-df2.iloc[i*2, 1]
        score_diff_2 = df1.iloc[i*2+1, 1]-df2.iloc[i*2+1, 1]
        score_mean = (score_diff_1 + score_diff_2) / 2.0
        score_var = (score_diff_1 - score_mean) ** 2 + (score_diff_2 - score_mean) ** 2
        rmse_differences.extend([score_diff_1**2, score_diff_2**2])
        rmse_variances.append(score_var)

    rmse_numerator = sum(rmse_differences)
    rmse_denominator = 2 * (sum(rmse_variances))
    rmse_f_stat = rmse_numerator / rmse_denominator
    rmse_p_value = scipy.stats.f.sf(rmse_f_stat, 10, 5)

    #Test R2
    for i in range(5):
        #first column==0 is R2
        #second column==1 is RMSE
        score_diff_1 = df1.iloc[i*2, 0]-df2.iloc[i*2, 0]
        score_diff_2 = df1.iloc[i*2+1, 0]-df2.iloc[i*2+1, 0]
        score_mean = (score_diff_1 + score_diff_2) / 2.0
        score_var = (score_diff_1 - score_mean) ** 2 + (score_diff_2 - score_mean) ** 2
        r2_differences.extend([score_diff_1**2, score_diff_2**2])
        r2_variances.append(score_var)

    r2_numerator = sum(r2_differences)
    r2_denominator = 2 * (sum(r2_variances))
    r2_f_stat = r2_numerator / r2_denominator
    r2_p_value = scipy.stats.f.sf(r2_f_stat, 10, 5)

    return float(r2_f_stat), float(r2_p_value),float(rmse_f_stat), float(rmse_p_value)

In [ ]:
import pandas as pd
import numpy as np

# Test Overall

In [ ]:
#here you have to define the right csv file which stores the perfromance values
#'tfidf','glove200', '1024elmo', 'berttwitter','openai
# compare per nlp with the corresponding best model
# check with the file anme
#here you have to define the models(e.g,SVM)
SWB_list=['SWBcompound',
          'SWBTextBlob_score',
          'SWBNNtfidf',
          'SWBNNglove200',
          'SWBSVM1024elmo',
          'SWBSVMberttwitter',
          'SWBNNopenai']


#here you have to define the models(e.g,SVM)
PA_list=['Positive_affectpos',
         'PANNtfidf',
         'PANNglove200',
         'PASVM1024elmo',
         'PASVMberttwitter',
         'PANNopenai'
      ]

#here you have to define the models(e.g,SVM)
NA_list=['Negative_affectneg',
         'NANNtfidf',
         'NAelasticglove200',
         'NASVM1024elmo',
         'NAelasticberttwitter',
         'NASVMopenai'
        ]

In [ ]:
for item in SWB_list:
  print(item)
  df1 = pd.read_csv(f"/content/drive/MyDrive/codeAndData/data/Test_results/{item}.csv")
  first_column_mean = df1.iloc[:, 0].mean()
  second_column_mean = df1.iloc[:, 1].mean()
  print(f"Mean of the first column: {first_column_mean}")
  print(f"Mean of the second column: {second_column_mean}")





In [ ]:
def combinate_nlp_tests(list_name):

  from itertools import combinations
  from statsmodels.stats.multitest import multipletests

  # Generate all possible combinations of two items from SWB_list
  combinations_list = list(combinations(list_name, 2))
  df_total = pd.DataFrame(combinations_list, columns=['Combination1', 'Combination2'])
  #crete columns to store f and p
  df_total['r2_f']=np.nan
  df_total['r2_p']=np.nan
  df_total['rmse_f']=np.nan
  df_total['rmse_p']=np.nan

  for i in range(len(combinations_list)):
    #the rule of file name SWBSVMtfidf
    df1 = pd.read_csv(f"/content/drive/MyDrive/codeAndData/data/Test_results/{df_total.iloc[i,0]}.csv")
    df2 = pd.read_csv(f"/content/drive/MyDrive/codeAndData/data/Test_results/{df_total.iloc[i,1]}.csv")
    r2_f_stat, r2_p_value, rmse_f_stat, rmse_p_value = ftest_5x2cv(df1,df2)
    df_total.iloc[i,2]=r2_f_stat
    df_total.iloc[i,3]=r2_p_value
    df_total.iloc[i,4]=rmse_f_stat
    df_total.iloc[i,5]=rmse_p_value

 # the folowing code is for holm correction
     # Get the p-values columns
  p_values_list_r2 = df_total['r2_p'].tolist()
  p_values_list_rmse = df_total['rmse_p'].tolist()


  rejected_r2, corrected_p_values_r2, _, _ = multipletests(p_values_list_r2, method='holm', alpha=0.05)
  rejected_rmse, corrected_p_values_rmse, _, _ = multipletests(p_values_list_rmse, method='holm', alpha=0.05)

  df_total['corrected_p_values_r2'] = corrected_p_values_r2
  df_total['corrected_p_values_rmse'] = corrected_p_values_rmse

  df_total['rejected_r2'] = rejected_r2
  df_total['rejected_rmse'] = rejected_rmse

  print(df_total.round(4))
  return df_total





In [ ]:
#Test total SWB
nlp_df_SWB=combinate_nlp_tests(SWB_list)

In [ ]:
nlp_df_SWB

In [ ]:
# Define the path for saving the Excel file
excel_path = '/content/drive/MyDrive/codeAndData/data/Test_results/nlp_test_SWB.xlsx'
# Save means DataFrame to Excel
nlp_df_SWB.to_excel(excel_path, index=False)

In [ ]:
#Test PA

In [ ]:
nlp_df_PA=combinate_nlp_tests(PA_list)

In [ ]:
nlp_df_PA

In [ ]:
# Define the path for saving the Excel file
excel_path = '/content/drive/MyDrive/codeAndData/data/Test_results/nlp_df_PA.xlsx'
# Save means DataFrame to Excel
nlp_df_PA.to_excel(excel_path, index=False)

In [ ]:
#Test NA

In [ ]:
nlp_df_NA=combinate_nlp_tests(NA_list)

In [ ]:
nlp_df_NA

In [ ]:
# Define the path for saving the Excel file
excel_path = '/content/drive/MyDrive/codeAndData/data/Test_results/nlp_df_NA.xlsx'
# Save means DataFrame to Excel
nlp_df_NA.to_excel(excel_path, index=False)